# Market Zone Coupling in the ASSUME Framework

Welcome to the **Market Zone Coupling** tutorial for the ASSUME framework. In this workshop, we will guide you through understanding how market zone coupling is implemented within the ASSUME simulation environment. By the end of this tutorial, you will gain insights into the internal mechanisms of the framework, including how different market zones interact, how constraints are managed, how bids are assigned, and how market prices are extracted.

**We will cover the following topics:**

1. **Introduction to Market Zone Coupling**
2. **Setting Up the ASSUME Framework for Market Zone Coupling**
3. **Understanding the Market Clearing Optimization**
4. **Creating Exemplary Input Files for Market Zone Coupling**
   - 4.1. Defining Buses and Zones
   - 4.2. Configuring Transmission Lines
   - 4.3. Setting Up Power Plant and Demand Units
   - 4.4. Preparing Demand Forecast Data
5. **Mimicking the Market Clearing Process**
   - 5.1. Calculating the Incidence Matrix
   - 5.2. Implementing the Simplified Market Clearing Function
   - 5.3. Running the Market Clearing Simulation
6. **Integrating with ASSUME**
7. **Analyzing the Results**

Let's get started!

## 1. Introduction to Market Zone Coupling

**Market Zone Coupling** is a mechanism that enables different geographical zones within an electricity market to interact and trade energy seamlessly. By coupling market zones, we can simulate more realistic and complex market dynamics, considering factors like transmission constraints, regional demand and supply variations, and cross-zone trading.

In the ASSUME framework, market zone coupling involves:

- **Defining Multiple Market Zones:** Segmenting the market into distinct zones based on geographical or operational criteria.
- **Establishing Connections Between Zones:** Setting up transmission lines that allow energy flow between different market zones.
- **Configuring the Market Clearing Process:** Adjusting the market clearing algorithm to account for interactions and constraints across zones.

This tutorial will walk you through each of these steps, providing code examples and configuration guidelines to help you set up market zone coupling effectively.

## 2. Setting Up the ASSUME Framework for Market Zone Coupling

Before diving into market zone coupling, ensure that you have the ASSUME framework installed and set up correctly. If you haven't done so already, follow the steps below to install the ASSUME core package and clone the repository containing predefined scenarios.

**Note:** If you already have the ASSUME framework installed and the repository cloned, you can skip executing the following code cells.

In [ ]:
# Install the ASSUME framework
# !pip install assume-framework

# Install Plotly if not already installed
# !pip install plotly

In [ ]:
# Clone the ASSUME repository containing predefined scenarios
# !git clone https://github.com/assume-framework/assume.git assume-repo

Let's also import some basic libraries that we will use throughout the tutorial.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# import plotly for visualization
import plotly.graph_objects as go
import seaborn as sns

# Function to display DataFrame in Jupyter
from IPython.display import display

**Select Input Files Path:**

Depending on whether you're using Google Colab or a local environment, the input file paths may vary. The following code snippet helps differentiate between these environments and sets the appropriate input paths.

In [ ]:
import importlib.util

# Check if 'google.colab' is available
IN_COLAB = importlib.util.find_spec("google.colab") is not None

colab_inputs_path = "assume-repo/examples/inputs"
local_inputs_path = "../inputs"

inputs_path = colab_inputs_path if IN_COLAB else local_inputs_path

print(f"Using inputs path: {inputs_path}")

## 3. Understanding the Market Clearing Optimization

Market clearing is a crucial component of electricity market simulations. It involves determining the optimal dispatch of supply and demand bids to maximize social welfare while respecting network constraints.

In the context of market zone coupling, the market clearing process must account for:

- **Connection Between Zones:** Transmission lines that allow energy flow between different market zones.
- **Constraints:** Limits on transmission capacities and ensuring energy balance within and across zones.
- **Bid Assignment:** Properly assigning bids to their respective zones and considering cross-zone trading.
- **Price Extraction:** Determining market prices for each zone based on the cleared bids and network constraints.

The ASSUME framework uses Pyomo to formulate and solve the market clearing optimization problem. Below is a simplified version of the market clearing function, highlighting key components related to zone coupling.

In [ ]:
# Display a simplified version of the market clearing optimization function
import pyomo.environ as pyo
from pyomo.opt import SolverFactory, TerminationCondition


def simplified_market_clearing_opt(orders, market_products, nodes, incidence_matrix):
    """
    Simplified market clearing optimization focusing on zone coupling.

    Args:
        orders (dict): Dictionary of orders with bid_id as keys.
        market_products (list): List of MarketProduct tuples.
        nodes (list): List of market zones.
        incidence_matrix (dict): Transmission capacity between zones.

    Returns:
        model (ConcreteModel): The solved Pyomo model.
        results (SolverResults): The solver results.
    """

    model = pyo.ConcreteModel()
    # define duals suffix
    model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

    # Define the set of time periods
    model.T = pyo.Set(initialize=[mp[0] for mp in market_products], doc="timesteps")
    # Define the set of zones (nodes)
    model.nodes = pyo.Set(initialize=nodes, doc="nodes")

    # Decision variables for bid acceptance ratios (0 to 1)
    model.x = pyo.Var(
        orders.keys(),
        domain=pyo.NonNegativeReals,
        bounds=(0, 1),
        doc="bid_acceptance_ratio",
    )

    # Decision variables for power flows between zones at each time period
    model.flows = pyo.Var(
        model.T, model.nodes, model.nodes, domain=pyo.Reals, doc="power_flows"
    )

    # Energy balance constraints for each zone and time period
    def energy_balance_rule(model, node, t):
        """
        Ensures that for each zone and time period, the total supply minus demand plus imports minus exports equals zero.
        """
        # Sum of accepted bid volumes in the zone at time t
        supply = sum(
            orders[o]["volume"] * model.x[o]
            for o in orders
            if orders[o]["node"] == node and orders[o]["time"] == t
        )
        # Sum of power flows into the zone
        imports = sum(
            model.flows[t, other_node, node]
            for other_node in nodes
            if other_node != node
        )
        # Sum of power flows out of the zone
        exports = sum(
            model.flows[t, node, other_node]
            for other_node in nodes
            if other_node != node
        )
        # Energy balance: supply + imports - exports = 0
        return supply + imports - exports == 0

    # Apply the energy balance rule to all zones and time periods
    model.energy_balance = pyo.Constraint(
        model.nodes, model.T, rule=energy_balance_rule
    )

    # Transmission constraints based on the incidence matrix
    if incidence_matrix is not None:

        def transmission_rule(model, t, node1, node2):
            """
            Limits the power flow between two zones based on transmission capacity.
            """
            capacity = incidence_matrix[node1].get(node2, 0)
            return (-capacity, model.flows[t, node1, node2], capacity)

        # Apply the transmission constraints to all possible flows
        model.transmission_constraints = pyo.Constraint(
            model.T, model.nodes, model.nodes, rule=transmission_rule
        )

    # Objective: Minimize total cost (sum of bid prices multiplied by accepted volumes)
    model.objective = pyo.Objective(
        expr=sum(orders[o]["price"] * orders[o]["volume"] * model.x[o] for o in orders),
        sense=pyo.minimize,
        doc="Total Cost Minimization",
    )

    # Choose the solver (GLPK is used here for simplicity)
    solver = SolverFactory("glpk")
    results = solver.solve(model)

    market_clearing_prices = {}
    for node in nodes:
        market_clearing_prices[node] = {
            t: pyo.value(model.dual[model.energy_balance[node, t]]) for t in model.T
        }
    # Check if the solver found an optimal solution
    if results.solver.termination_condition != TerminationCondition.optimal:
        raise Exception("Solver did not find an optimal solution.")

    return model, results

The above function is a simplified representation focusing on the essential aspects of market zone coupling. In the following sections, we will delve deeper into creating input files and mimicking the market clearing process using this function to understand the inner workings of the ASSUME framework.

## 4. Creating Exemplary Input Files for Market Zone Coupling

To implement market zone coupling, users need to prepare specific input files that define the network's structure, units, and demand profiles. Below, we will guide you through creating the necessary DataFrames for buses, transmission lines, power plant units, demand units, and demand forecasts.

### 4.1. Defining Buses and Zones

**Buses** represent nodes in the network where energy can be injected or withdrawn. Each bus is assigned to a **zone**, which groups buses into market areas. This zoning facilitates market coupling by managing interactions between different market regions.

In [ ]:
# Define the buses DataFrame with three nodes and two zones
buses = pd.DataFrame(
    {
        "name": ["north_1", "north_2", "south"],
        "v_nom": [380.0, 380.0, 380.0],
        "zone_id": ["DE_1", "DE_1", "DE_2"],
        "x": [10.0, 9.5, 11.6],
        "y": [54.0, 53.5, 48.1],
    }
)

# Display the buses DataFrame
print("Buses DataFrame:")
print(buses)

**Explanation:**

- **name:** Identifier for each bus (`north_1`, `north_2`, and `south`).
- **v_nom:** Nominal voltage level (in kV) for all buses.
- **zone_id:** Identifier for the market zone to which the bus belongs (`DE_1` for north buses and `DE_2` for the south bus).
- **x, y:** Geographical coordinates (optional, can be used for mapping or spatial analyses).

### 4.2. Configuring Transmission Lines

**Transmission Lines** connect buses, allowing energy to flow between them. Each line has a specified capacity and electrical parameters.

In [ ]:
# Define the transmission lines DataFrame with three lines
lines = pd.DataFrame(
    {
        "name": ["Line_N1_S", "Line_N2_S", "Line_N1_N2"],
        "bus0": ["north_1", "north_2", "north_1"],
        "bus1": ["south", "south", "north_2"],
        "s_nom": [10000.0, 10000.0, 5000.0],  # Increased capacities for clarity
        "x": [0.01, 0.01, 0.01],
        "r": [0.001, 0.001, 0.001],
    }
)

# Display the lines DataFrame
print("Transmission Lines DataFrame:")
print(lines)

**Explanation:**

- **name:** Identifier for each transmission line (`Line_N1_S`, `Line_N2_S`, and `Line_N1_N2`).
- **bus0, bus1:** The two buses that the line connects.
- **s_nom:** Nominal apparent power capacity of the line (in MVA).
- **x:** Reactance of the line (in per unit).
- **r:** Resistance of the line (in per unit).

### 4.3. Setting Up Power Plant and Demand Units

**Power Plant Units** represent energy generation sources, while **Demand Units** represent consumption. Each unit is associated with a specific bus (node) and has operational parameters that define its behavior in the market.

In [ ]:
# Define the total number of units
num_units = 30  # Reduced for simplicity

# Generate the 'name' column: Unit 1 to Unit 30
names = [f"Unit {i}" for i in range(1, num_units + 1)]

# All other columns with constant values
technology = ["nuclear"] * num_units
bidding_nodal = ["naive_eom"] * num_units
fuel_type = ["uranium"] * num_units
emission_factor = [0.0] * num_units
max_power = [1000.0] * num_units
min_power = [0.0] * num_units
efficiency = [0.3] * num_units

# Generate 'additional_cost':
# - North units (1-15): 5 to 19
# - South units (16-30): 20 to 34
additional_cost = list(range(5, 5 + num_units))

# Initialize 'node' and 'unit_operator' lists
node = []
unit_operator = []

for i in range(1, num_units + 1):
    if 1 <= i <= 15:
        node.append("north_1")  # All north units connected to 'north_1'
        unit_operator.append("Operator North")
    else:
        node.append("south")  # All south units connected to 'south'
        unit_operator.append("Operator South")

# Create the DataFrame
powerplant_units = pd.DataFrame(
    {
        "name": names,
        "technology": technology,
        "bidding_nodal": bidding_nodal,
        "fuel_type": fuel_type,
        "emission_factor": emission_factor,
        "max_power": max_power,
        "min_power": min_power,
        "efficiency": efficiency,
        "additional_cost": additional_cost,
        "node": node,
        "unit_operator": unit_operator,
    }
)

# Display the powerplant_units DataFrame
print("Power Plant Units DataFrame:")
print(powerplant_units)

In [ ]:
# Define the demand units DataFrame
demand_units = pd.DataFrame(
    {
        "name": ["demand_north_1", "demand_north_2", "demand_south"],
        "technology": ["inflex_demand"] * 3,
        "bidding_zonal": ["naive_eom"] * 3,
        "max_power": [100000, 100000, 100000],
        "min_power": [0, 0, 0],
        "unit_operator": ["eom_de"] * 3,
        "node": ["north_1", "north_2", "south"],
    }
)

# Display the demand_units DataFrame
print("Demand Units DataFrame:")
print(demand_units)

**Explanation:**

- **Power Plant Units:**
    - **name:** Identifier for each power plant unit (`Unit 1` to `Unit 30`).
    - **technology:** Type of technology (`nuclear` for all units).
    - **bidding_nodal:** Bidding strategy used (`naive_eom` for all units).
    - **fuel_type:** Type of fuel used (`uranium` for all units).
    - **emission_factor:** Emissions per unit of energy produced (`0.0` for all units).
    - **max_power, min_power:** Operational power limits (`1000.0` MW max, `0.0` MW min for all units).
    - **efficiency:** Conversion efficiency (`0.3` for all units).
    - **additional_cost:** Additional operational costs (`5` to `34`, with southern units being more expensive).
    - **node:** The bus (zone) to which the unit is connected (`north_1` for units `1-15`, `south` for units `16-30`).
    - **unit_operator:** Operator responsible for the unit (`Operator North` for northern units, `Operator South` for southern units).

- **Demand Units:**
    - **name:** Identifier for each demand unit (`demand_north_1`, `demand_north_2`, and `demand_south`).
    - **technology:** Type of demand (`inflex_demand` for all units).
    - **bidding_zonal:** Bidding strategy used (`naive_eom` for all units).
    - **max_power, min_power:** Operational power limits (`100000` MW max, `0` MW min for all units).
    - **unit_operator:** Operator responsible for the unit (`eom_de` for all units).
    - **node:** The bus (zone) to which the unit is connected (`north_1`, `north_2`, and `south`).

### 4.4. Preparing Demand Forecast Data

**Demand Forecast Data** provides the expected electricity demand for each demand unit over time. This data is essential for simulating how demand varies and affects market dynamics.

In [ ]:
# Define the demand forecast DataFrame
demand_df = pd.DataFrame(
    {
        "datetime": [
            "2019-01-01 00:00:00",
            "2019-01-01 01:00:00",
            "2019-01-01 02:00:00",
            "2019-01-01 03:00:00",
            "2019-01-01 04:00:00",
            "2019-01-01 05:00:00",
            "2019-01-01 06:00:00",
            "2019-01-01 07:00:00",
            "2019-01-01 08:00:00",
            "2019-01-01 09:00:00",
            "2019-01-01 10:00:00",
            "2019-01-01 11:00:00",
            "2019-01-01 12:00:00",
            "2019-01-01 13:00:00",
            "2019-01-01 14:00:00",
            "2019-01-01 15:00:00",
            "2019-01-01 16:00:00",
            "2019-01-01 17:00:00",
            "2019-01-01 18:00:00",
            "2019-01-01 19:00:00",
            "2019-01-01 20:00:00",
            "2019-01-01 21:00:00",
            "2019-01-01 22:00:00",
            "2019-01-01 23:00:00",
        ],
        "demand_north_1": [
            2400.0,
            2800.0,
            3200.0,
            3600.0,
            4000.0,
            4400.0,
            4800.0,
            5200.0,
            5600.0,
            6000.0,
            6400.0,
            6800.0,
            7200.0,
            7600.0,
            8000.0,
            8400.0,
            8800.0,
            9200.0,
            9600.0,
            10000.0,
            10400.0,
            10800.0,
            11200.0,
            11600.0,
        ],
        "demand_north_2": [
            2400.0,
            2800.0,
            3200.0,
            3600.0,
            4000.0,
            4400.0,
            4800.0,
            5200.0,
            5600.0,
            6000.0,
            6400.0,
            6800.0,
            7200.0,
            7600.0,
            8000.0,
            8400.0,
            8800.0,
            9200.0,
            9600.0,
            10000.0,
            10400.0,
            10800.0,
            11200.0,
            11600.0,
        ],
        "demand_south": [
            17400.0,
            16800.0,
            16200.0,
            15600.0,
            15000.0,
            14400.0,
            13800.0,
            13200.0,
            12600.0,
            12000.0,
            11400.0,
            10800.0,
            10200.0,
            9600.0,
            9000.0,
            8400.0,
            7800.0,
            7200.0,
            6600.0,
            6000.0,
            5400.0,
            4800.0,
            4200.0,
            3600.0,
        ],
    }
)

# Convert the 'datetime' column to datetime objects and set as index
demand_df["datetime"] = pd.to_datetime(demand_df["datetime"])
demand_df.set_index("datetime", inplace=True)

# Display the demand_df DataFrame
print("Demand Forecast DataFrame:")
print(demand_df.head())

**Explanation:**

- **datetime:** Timestamp for each demand forecast.
- **demand_north_1, demand_north_2, demand_south:** Forecasted demand values for each respective demand unit.

**Note:** The demand timeseries has been designed to be fulfillable by the defined power plants in both zones.

## 5. Mimicking the Market Clearing Process

With the input files prepared, we can now mimic the market clearing process using the simplified market clearing function. This will help us understand how different market zones interact, how constraints are managed, how bids are assigned, and how market prices are extracted.

### 5.1. Calculating the Incidence Matrix

The **Incidence Matrix** represents the transmission capacities between different market zones. It is calculated as the sum of the capacities of transmission lines connecting each pair of zones. This matrix is crucial for enforcing transmission constraints during the market clearing process.

**Note:** The method of calculating the incidence matrix by simply summing line capacities is a simplified approach. In real-world scenarios, more sophisticated methods are used to accurately represent the network's behavior and constraints. This approach will be extended in future implementations to better reflect real-world complexities.

In [ ]:
# Define market products (time periods)
market_products = []
for timestamp in demand_df.index:
    market_products.append(
        (
            timestamp,  # Start time
            timestamp + pd.Timedelta(hours=1),  # End time
            1,  # Only_hours flag (for simplicity)
        )
    )

# Define nodes (zones)
nodes = buses["zone_id"].unique().tolist()

# Calculate the incidence matrix by summing the capacities of transmission lines between zones
incidence_matrix = {zone: {} for zone in nodes}

for _, line in lines.iterrows():
    # Get zones for each end of the transmission line
    zone0 = buses.loc[buses["name"] == line["bus0"], "zone_id"].values[0]
    zone1 = buses.loc[buses["name"] == line["bus1"], "zone_id"].values[0]

    if zone0 != zone1:
        # Add capacity to zone0 -> zone1
        if zone1 not in incidence_matrix[zone0]:
            incidence_matrix[zone0][zone1] = 0
        incidence_matrix[zone0][zone1] += line["s_nom"]

        # Add capacity to zone1 -> zone0 (assuming bidirectional)
        if zone0 not in incidence_matrix[zone1]:
            incidence_matrix[zone1][zone0] = 0
        incidence_matrix[zone1][zone0] += line["s_nom"]

# Convert lower triangle values to negative to indicate opposite direction
for i, zone0 in enumerate(nodes):
    for j, zone1 in enumerate(nodes):
        if i > j and zone1 in incidence_matrix[zone0]:
            incidence_matrix[zone0][zone1] = -incidence_matrix[zone0][zone1]

# Display the calculated incidence matrix
print("Calculated Incidence Matrix between Zones:")
print(pd.DataFrame(incidence_matrix))

**Explanation:**

- **Nodes (Zones):** Extracted from the `buses` DataFrame (`DE_1` and `DE_2`).
- **Transmission Lines:** Iterated over to sum their capacities between different zones.
- **Bidirectional Flow Assumption:** Transmission capacities are added in both directions (`DE_1 -> DE_2` and `DE_2 -> DE_1`).
- **Lower Triangle Negative Values:** To indicate the opposite direction of power flow, capacities in the lower triangle of the matrix are converted to negative values.

**Sample Output:**

```
Calculated Incidence Matrix between Zones:
       DE_1    DE_2
DE_1     0  20000
DE_2 -20000      0
```

This output indicates that there is a total transmission capacity of 20,000 MVA from `DE_1` to `DE_2` and vice versa, based on the sum of the capacities of the transmission lines connecting these zones.

### 5.2. Implementing the Simplified Market Clearing Function

We will use the `simplified_market_clearing_opt` function defined earlier to perform the market clearing. This function takes in the orders, market products, zones (nodes), and the incidence matrix to determine the optimal bid acceptances and power flows between zones.

In [ ]:
# Prepare the orders dictionary based on powerplant_units and demand_units

# Initialize orders dictionary
orders = {}

# Add power plant bids
for _, row in powerplant_units.iterrows():
    bid_id = row["name"]
    for timestamp in demand_df.index:
        orders[f"{bid_id}_{timestamp}"] = {
            "price": row["additional_cost"],  # Assuming additional_cost as bid price
            "volume": row["max_power"],  # Assuming max_power as bid volume
            "node": row["node"],
            "time": timestamp,
        }

# Add demand bids
for _, row in demand_units.iterrows():
    bid_id = row["name"]
    for timestamp in demand_df.index:
        orders[f"{bid_id}_{timestamp}"] = {
            "price": 100,  # Demand bids with high price
            "volume": -demand_df.loc[
                timestamp, row["name"]
            ],  # Negative volume for demand
            "node": row["node"],
            "time": timestamp,
        }

# Display a sample order
print("\nSample Order:")
print(orders["Unit 1_2019-01-01 00:00:00"])

**Explanation:**

- **Power Plant Bids:** Each power plant unit submits a bid for each time period with its `additional_cost` as the bid price and `max_power` as the bid volume. Units in the north (`DE_1`) are cheaper (`additional_cost` ranging from 5 to 19) compared to southern units (`DE_2`) which are more expensive (`additional_cost` ranging from 20 to 34).
- **Demand Bids:** Each demand unit submits a bid for each time period with zero price and negative volume representing the demand.

### 5.3. Running the Market Clearing Simulation

We will conduct two simulations:

1. **Simulation 1:** Transmission capacities between `DE_1` (north) and `DE_2` (south) are **zero**.
2. **Simulation 2:** Transmission capacities between `DE_1` (north) and `DE_2` (south) are **present**.

#### Simulation 1: Zero Transmission Capacity Between Zones

In [ ]:
print("### Simulation 1: Zero Transmission Capacity Between Zones")

# Define nodes (zones)
nodes_sim1 = nodes.copy()

# Define the incidence matrix as a dictionary with zero transmission capacity
incidence_matrix_sim1 = {
    "DE_1": {"DE_2": 0.0},  # Zero capacity from DE_1 to DE_2
    "DE_2": {"DE_1": 0.0},  # Zero capacity from DE_2 to DE_1
}

# Display the incidence matrix for Simulation 1
print("Incidence Matrix for Simulation 1 (Zero Transmission Capacity):")
display(pd.DataFrame(incidence_matrix_sim1))

The orders before being sent to the market clearing are first preprocessed. During this preprocessing the node_id from the bids is matched with the zone_id from the buses DataFrame. The bids are then sent to the market clearing function.

In [ ]:
# create a mapping from node_id to zone_id
node_mapping = buses.set_index("name")["zone_id"].to_dict()

# Create a new dictionary with mapped zone IDs
orders_mapped = {}
for bid_id, bid in orders.items():
    original_node = bid["node"]
    mapped_zone = node_mapping.get(
        original_node, original_node
    )  # Default to original_node if not found
    orders_mapped[bid_id] = {
        "price": bid["price"],
        "volume": bid["volume"],
        "node": mapped_zone,  # Replace bus with zone ID
        "time": bid["time"],
    }

display(pd.DataFrame(orders_mapped).T.head())

Now we can run the market clearing

In [ ]:
# Run the simplified market clearing for Simulation 1
model_sim1, results_sim1 = simplified_market_clearing_opt(
    orders_mapped, market_products, nodes_sim1, incidence_matrix_sim1
)

#### Simulation 2: Transmission Capacity Present Between Zones

In [ ]:
print("### Simulation 2: Transmission Capacity Present Between Zones")

# Define the incidence matrix as a dictionary with non-zero transmission capacity
incidence_matrix_sim2 = {
    "DE_1": {"DE_2": 20000.0},  # Transmission capacity from DE_1 to DE_2 in MVA
    "DE_2": {"DE_1": 20000.0},  # Transmission capacity from DE_2 to DE_1 in MVA
}

# Display the incidence matrix for Simulation 2
print("Incidence Matrix for Simulation 2 (With Transmission Capacity):")
display(pd.DataFrame(incidence_matrix_sim2))

# since the orders are already mapped to zones, we can directly use the orders_mapped dictionary

# Run the simplified market clearing for Simulation 2
model_sim2, results_sim2 = simplified_market_clearing_opt(
    orders_mapped, market_products, nodes_sim1, incidence_matrix_sim2
)

### 5.4. Extracting and Interpreting the Results

After running both simulations, we can extract the results to understand how the presence or absence of transmission capacity affects bid acceptances and power flows between zones.

In [ ]:
# Function to extract accepted bids and calculate clearing prices
# Function to extract accepted bids, power flows, and market clearing prices using dual variables
def extract_results(model, orders, nodes):
    # Extract accepted bid ratios
    accepted_bids = {}
    for o in model.x:
        acceptance_ratio = pyo.value(model.x[o])
        if acceptance_ratio > 0:
            accepted_bids[o] = acceptance_ratio

    # Extract power flows between zones for each time period
    power_flows = []
    for t in model.T:
        for node1 in nodes:
            for node2 in nodes:
                if node1 != node2:
                    flow = pyo.value(model.flows[t, node1, node2])
                    if flow != 0:
                        power_flows.append(
                            {
                                "time": t,
                                "from_zone": node1,
                                "to_zone": node2,
                                "flow_MW": flow,
                            }
                        )

    # Convert to DataFrame
    power_flows_df = pd.DataFrame(power_flows)

    # Extract market clearing prices from dual variables of energy balance constraints
    market_clearing_prices = {}
    for node in nodes:
        market_clearing_prices[node] = {
            t: pyo.value(model.dual[model.energy_balance[node, t]]) for t in model.T
        }

    # Convert clearing prices to DataFrame
    clearing_prices = []
    for node in market_clearing_prices:
        for t in market_clearing_prices[node]:
            clearing_prices.append(
                {
                    "zone": node,
                    "time": t,
                    "clearing_price": market_clearing_prices[node][t],
                }
            )

    clearing_prices_df = pd.DataFrame(clearing_prices)

    return accepted_bids, power_flows_df, clearing_prices_df

In [ ]:
# Extract results for Simulation 1
accepted_bids_sim1, power_flows_df_sim1, clearing_prices_df_sim1 = extract_results(
    model_sim1, orders, nodes_sim1
)

print("Simulation 1: Power Flows Between Zones")
display(power_flows_df_sim1.head())

print("Simulation 1: Clearing Prices per Zone and Time")
display(clearing_prices_df_sim1.loc[clearing_prices_df_sim1["zone"] == "DE_1"].head())
display(clearing_prices_df_sim1.loc[clearing_prices_df_sim1["zone"] == "DE_2"].head())

# Extract results for Simulation 2
accepted_bids_sim2, power_flows_df_sim2, clearing_prices_df_sim2 = extract_results(
    model_sim2, orders, nodes_sim1
)

print("Simulation 2: Power Flows Between Zones")
display(power_flows_df_sim2.head())

print("Simulation 2: Clearing Prices per Zone and Time")
display(clearing_prices_df_sim2.loc[clearing_prices_df_sim2["zone"] == "DE_1"].head())
display(clearing_prices_df_sim2.loc[clearing_prices_df_sim2["zone"] == "DE_2"].head())

**Explanation:**

- **Accepted Bids:** Shows which bids were accepted in each simulation and the ratio at which they were accepted.
- **Power Flows:** Indicates the amount of energy transmitted between zones. In Simulation 1, with zero transmission capacity, there should be no power flows between `DE_1` and `DE_2`. In Simulation 2, with transmission capacity present, power flows can occur between zones.
- **Clearing Prices:** Represents the average bid price in each zone at each time period. Comparing prices across simulations can reveal the impact of transmission capacity on market prices.

### 5.5. Comparing Simulations

To better understand the impact of transmission capacity, let's compare the key results from both simulations.

In [ ]:
# Initialize the Plotly figure
fig = go.Figure()

# Iterate over each zone to plot clearing prices for both simulations
for zone in nodes_sim1:
    # Filter data for the current zone and Simulation 1
    zone_prices_sim1 = clearing_prices_df_sim1[clearing_prices_df_sim1["zone"] == zone]
    # Filter data for the current zone and Simulation 2
    zone_prices_sim2 = clearing_prices_df_sim2[clearing_prices_df_sim2["zone"] == zone]

    # Add trace for Simulation 1
    fig.add_trace(
        go.Scatter(
            x=zone_prices_sim1["time"],
            y=zone_prices_sim1["clearing_price"],
            mode="lines",
            name=f"{zone} - Sim1",
            line=dict(dash="dash"),  # Dashed line for Simulation 1
        )
    )

    # Add trace for Simulation 2
    fig.add_trace(
        go.Scatter(
            x=zone_prices_sim2["time"],
            y=zone_prices_sim2["clearing_price"],
            mode="lines",
            name=f"{zone} - Sim2",
            line=dict(dash="solid"),  # Solid line for Simulation 2
        )
    )

# Update layout for better aesthetics and interactivity
fig.update_layout(
    title="Clearing Prices per Zone Over Time: Simulation 1 vs Simulation 2",
    xaxis_title="Time",
    yaxis_title="Clearing Price",
    legend_title="Simulation",
    xaxis=dict(
        tickangle=45,
        type="date",  # Ensure the x-axis is treated as dates
    ),
    hovermode="x unified",  # Unified hover for better comparison
    template="plotly_white",  # Clean white background
    width=1000,
    height=600,
)

# Display the interactive plot
fig.show()

In [ ]:
# Compare power flows
plt.figure(figsize=(14, 8))
sns.heatmap(
    power_flows_df_sim2.pivot_table(
        index="time", columns=["from_zone", "to_zone"], values="flow_MW", fill_value=0
    ),
    annot=True,
    fmt=".1f",
    cmap="coolwarm",
)
plt.xlabel("Transmission Lines (From Zone -> To Zone)")
plt.ylabel("Time")
plt.title("Power Flows Between Zones Over Time: Simulation 2")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

**Explanation:**

- **Number of Accepted Bids:** Comparing the total number of accepted bids in both simulations can show how transmission capacity affects the ability to dispatch more units.
- **Clearing Prices Plot:** By plotting the clearing prices for each zone across both simulations, we can observe how the presence of transmission capacity influences price convergence or divergence between zones.
- **Power Flows Heatmap:** Visualizing power flows in Simulation 2 demonstrates how energy is transmitted between zones when transmission capacity is available.

## 6. Integrating with ASSUME

In a real-world scenario, the ASSUME framework would handle the reading of CSV files and the configuration of the simulation through configuration files. For the purpose of this tutorial, we've mimicked the market clearing process using DataFrames and a simplified function to illustrate the inner workings of market zone coupling.

To integrate this process within ASSUME:

- **Input Files:** Prepare CSV files for buses, lines, powerplant_units, demand_units, and demand forecasts as demonstrated in section 4.
- **Configuration File:** Define the market configuration, including zone identifiers and transmission capacities.
- **Running the Simulation:** Use ASSUME's built-in functions to load the CSV files, apply the configurations, and execute the simulation.

Refer to the ASSUME documentation for detailed instructions on configuring and running simulations with CSV input files and configuration settings.

## 7. Analyzing the Results

After running the market clearing simulation, it's essential to analyze the results to understand how market zone coupling influenced market dynamics. Below are key metrics and methods to analyze the simulation outcomes.

### 7.1. Accepted Bids

**Accepted Bids** indicate which power plant and demand bids were successfully dispatched in the market. Analyzing accepted bids helps in understanding supply and demand distribution across different zones.

In [ ]:
# Display accepted bids for Simulation 1
print("Simulation 1: Accepted Bids DataFrame:")
accepted_bids_df_sim1 = pd.DataFrame.from_dict(
    accepted_bids_sim1, orient="index", columns=["Acceptance_Ratio"]
)
accepted_bids_df_sim1.reset_index(inplace=True)
accepted_bids_df_sim1.rename(columns={"index": "Bid_ID"}, inplace=True)
display(accepted_bids_df_sim1.head())

# Display accepted bids for Simulation 2
print("Simulation 2: Accepted Bids DataFrame:")
accepted_bids_df_sim2 = pd.DataFrame.from_dict(
    accepted_bids_sim2, orient="index", columns=["Acceptance_Ratio"]
)
accepted_bids_df_sim2.reset_index(inplace=True)
accepted_bids_df_sim2.rename(columns={"index": "Bid_ID"}, inplace=True)
display(accepted_bids_df_sim2.head())

### 7.2. Power Flows Between Zones

**Power Flows** show the amount of energy transmitted between different zones. This helps in verifying that transmission constraints are respected and understanding cross-zone energy trading.

In [ ]:
# Display power flows for Simulation 1
print("Simulation 1: Power Flows Between Zones:")
display(power_flows_df_sim1.head())

# Display power flows for Simulation 2
print("Simulation 2: Power Flows Between Zones:")
display(power_flows_df_sim2.head())

### 7.3. Clearing Prices

**Clearing Prices** represent the market price in each zone at each time period. Comparing prices across zones can reveal how zone coupling affects market equilibrium and price convergence.

In [ ]:
# Display clearing prices for Simulation 1
print("Simulation 1: Clearing Prices per Zone and Time:")
display(clearing_prices_df_sim1.head())

# Display clearing prices for Simulation 2
print("Simulation 2: Clearing Prices per Zone and Time:")
display(clearing_prices_df_sim2.head())

### 7.4. Visualization (Optional)

Visualizing the results can provide a clearer understanding of the market dynamics. Below are examples of how to plot clearing prices and power flows.

In [ ]:
# Plot Clearing Prices for Each Zone Over Time
plt.figure(figsize=(12, 6))
for zone in nodes_sim1:
    zone_prices_sim1 = clearing_prices_df_sim1[clearing_prices_df_sim1["zone"] == zone]
    zone_prices_sim2 = clearing_prices_df_sim2[clearing_prices_df_sim2["zone"] == zone]
    plt.plot(
        zone_prices_sim1["time"],
        zone_prices_sim1["clearing_price"],
        label=f"{zone} - Sim1",
        linestyle="--",
    )
    plt.plot(
        zone_prices_sim2["time"],
        zone_prices_sim2["clearing_price"],
        label=f"{zone} - Sim2",
        linestyle="-",
    )
plt.xlabel("Time")
plt.ylabel("Clearing Price")
plt.title("Clearing Prices per Zone Over Time: Simulation 1 vs Simulation 2")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Explanation:**

- **Clearing Prices Plot:** Shows how market prices vary over time for each zone across both simulations. The dashed lines represent Simulation 1 (no transmission capacity), and the solid lines represent Simulation 2 (with transmission capacity). This visualization helps in observing how the presence of transmission capacity affects price convergence or divergence between zones.

In [ ]:
# Plot Power Flows Between Zones Over Time for Simulation 2
plt.figure(figsize=(14, 8))
sns.heatmap(
    power_flows_df_sim2.pivot_table(
        index="time", columns=["from_zone", "to_zone"], values="flow_MW", fill_value=0
    ),
    annot=True,
    fmt=".1f",
    cmap="coolwarm",
)
plt.xlabel("Transmission Lines (From Zone -> To Zone)")
plt.ylabel("Time")
plt.title("Power Flows Between Zones Over Time: Simulation 2")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

**Explanation:**

- **Power Flows Heatmap (Simulation 2):** Visualizes the amount of energy transmitted between zones over time when transmission capacity is present. The heatmap highlights periods of high or low cross-zone trading, demonstrating the impact of transmission capacity on energy distribution.

## Conclusion

In this tutorial, we explored how to mimic and understand market zone coupling within the ASSUME framework. We covered the following key steps:

1. **Introduction to Market Zone Coupling:** Understanding the concept and its significance in electricity market simulations.
2. **Setting Up the ASSUME Framework:** Installing the framework and preparing the environment for zone coupling.
3. **Understanding Market Clearing Optimization:** Grasping the fundamentals of the market clearing process with a focus on zone interactions.
4. **Creating Exemplary Input Files for Market Zone Coupling:** Defining buses, transmission lines, power plant units, demand units, and preparing demand forecasts.
5. **Mimicking the Market Clearing Process:** 
    - **Calculating the Incidence Matrix:** Summing transmission line capacities between zones to create the incidence matrix.
    - **Implementing the Simplified Market Clearing Function:** Using a simplified Pyomo model to perform market clearing.
    - **Running the Market Clearing Simulation:** Executing two simulations to observe the impact of transmission capacity.
6. **Integrating with ASSUME:** Outlining the steps to integrate the process within the ASSUME framework using CSV input files and configuration settings.
7. **Analyzing the Results:** Extracting and visualizing key metrics to assess the impact of market zone coupling.

**Key Takeaways:**

- **Incidence Matrix:** Represents the transmission capacities between different market zones. Calculated by summing the capacities of transmission lines connecting each pair of zones. While this method is simplified, it provides a foundational understanding of how zones interact within the market.

- **Simplified Market Clearing:** Demonstrates the core principles of market clearing, including bid acceptance and power flow management between zones.

- **Impact of Transmission Capacity:** The presence of transmission capacity allows for energy trading between zones, potentially reducing overall costs and balancing demand more effectively. Without transmission capacity, each zone must rely solely on its local generation, which may lead to higher costs if local generation is more expensive.

- **Limitations and Future Work:** The current method of calculating the incidence matrix by summing line capacities is limited and does not capture the full complexity of real-world transmission networks. Future implementations will enhance this approach to include more detailed network modeling, accounting for factors like line impedance, reactive power flows, and dynamic constraints.

By following this guide, you have successfully set up a simplified multi-zone electricity market simulation, enabling a deeper understanding of market dynamics and the role of zone coupling in balancing supply and demand. You can now extend this setup to include more zones, varying transmission capacities, and diverse bidding strategies to explore a wide range of market scenarios.

Thank you for participating in this workshop!